In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

# Models
from sklearn.model_selection import GridSearchCV
#from sklearn.metrics import confusion_matrix, plot_confusion_matrix, classification_report
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, precision_recall_curve, roc_curve, roc_auc_score
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from scipy.cluster.hierarchy import ward
from sklearn.mixture import GaussianMixture
import xgboost as xgb

In [4]:
# CSV 파일 불러오기
data = pd.read_csv('./dataset/combined_data.csv')


In [5]:
X = data[['x', 'y', 'z']]  # 입력 변수
y = data['position']           # 출력 변수


In [6]:
data['position'] = data['position'].astype(int) - 1

In [7]:
data.to_csv('zero_combined_data.csv', index=False)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)


In [7]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)


In [16]:
def hyperp_search(classifier, parameters, k, X_train, y_train, X_test, y_test):
    
    # Hyperparameter tuning via cross-validation
    gs = GridSearchCV(classifier, parameters, cv=k, scoring = 'accuracy', refit=True, verbose=0, n_jobs=-1) # refit=True refits the model with the best found parameters on the whole dataset
    gs = gs.fit(X_train, y_train)

    # Prediction on train and test sets with best model found
    best_model = gs.best_estimator_
    y_test_pred = best_model.predict(X_test)
    y_train_pred = best_model.predict(X_train)

    # Results display
    print("\n \033[1m ***Best result obtained*** \033[0m \n")
    print("accuracy (mean cross-validated score): %f using %s" % (gs.best_score_, gs.best_params_))

    print("\n \033[1m ***Scores obtained on train and test sets with best model*** \033[0m \n")
    print("f1          train %.3f   test %.3f" % (f1_score(y_train, y_train_pred, average='macro'), f1_score(y_test, y_test_pred, average='macro'))) 
    print("recall      train %.3f   test %.3f" % (recall_score(y_train, y_train_pred, average='macro'), recall_score(y_test, y_test_pred, average='macro'))) 
    print("precision   train %.3f   test %.3f" % (precision_score(y_train, y_train_pred, average='macro'), precision_score(y_test, y_test_pred, average='macro'))) 
    print("accuracy    train %.3f   test %.3f" % (accuracy_score(y_train, y_train_pred), accuracy_score(y_test, y_test_pred))) 

    print("\n \033[1m ***Classification report on test set*** \033[0m \n")
    print(classification_report(y_test, y_test_pred, target_names=['1', '2', '3', '4']))

    print("\n \033[1m ***Confusion matrix on test set*** \033[0m \n")
    ConfusionMatrixDisplay(best_model, X_test, y_test, display_labels=['1', '2', '3', '4'], values_format='', cmap='Reds')

    return best_model

In [18]:
classifier = KNeighborsClassifier()
parameters = {'n_neighbors':np.arange(3,50,1)}

best_model_knn_avg = hyperp_search(classifier, parameters, 5, X_train, y_train, X_test, y_test)

UnicodeEncodeError: 'ascii' codec can't encode characters in position 18-20: ordinal not in range(128)